In [2]:
# pip install pymssql
import pymssql
import pandas as pd

pd.options.display.max_columns = None

C:\Users\rmr\.conda\envs\class\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  


In [3]:
load_csv = True

In [4]:
if load_csv == True:
    df_sku = pd.read_csv('df_sku.csv')

In [5]:
if load_csv == False:
    sqluser = input('Enter SQL User')
    sqlpass = input(f'Enter Password for {sqluser}')
    ## instance a python db connection object- same form as psycopg2/python-mysql drivers also
    conn = pymssql.connect(server="192.168.254.13", user=sqluser,password=passwrd, port=1433)  # You can lookup the port number inside SQL server. 

    stmt = "SELECT \
            SKU_ID \
            ,UOM_ID \
            ,SalesCategoryID \
            ,cat.CategoryID \
            ,NACSCategoryID \
            ,Category \
            ,Description \
            ,LongDescription \
            ,ShortDescription \
            ,POSDescription \
            FROM AgilityPB.dbo.tbl_SKU sku \
            left outer join Agility_Net.dbo.tbl_Categories cat on sku.SalesCategoryID = cat.CategoryID"
    # Excute Query here
    df_sku = pd.read_sql(stmt,conn)
    df_sku.to_csv('df_sku.csv')

In [6]:
df_sku.head()

,Unnamed: 0,SKU_ID,UOM_ID,SalesCategoryID,CategoryID,NACSCategoryID,Category,Description,LongDescription,ShortDescription,POSDescription
0,0,1,119,40,40,140,General Merchandise,General Merchandise,1000 DRINKING GAMES IN A BOX,1000 DRINK GAME,1000 DRINKING GAMES
1,1,2,119,52,52,91,Salty Snacks,Salty Snacks Inv,2/$1 RED HOT SAUSAGE,MEAT SNACK RH,2/$1 RED HOT SAUSAGE
2,2,3,119,52,52,91,Salty Snacks,Salty Snacks Inv,20 COUNT VARIETY PACK,VARIETY 20CT,20 COUNT VARIETY PACK
3,3,4,119,52,52,91,Salty Snacks,Salty Snacks Inv,26 CT 100 CALORIE VARIETY PACK,VARIETY 100CAL,26CT 100 CALORIE VARIETY
4,4,5,119,52,52,91,Salty Snacks,Salty Snacks Inv,3 CHEESE MEDLEY NIBBLERS,PRETZEL 3 CHEESE,3 CHEESE MEDLEY NIBBLERS


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df_sku['LongDescription'], df_sku['Category'], train_size=0.8)

<h1>Naive Bayes Classifier</h1>

In [9]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
                     ])

text_clf.fit(X_train, y_train)


predicted = text_clf.predict(X_test)

print(metrics.classification_report(y_test, predicted))

                              precision    recall  f1-score   support

          Alternative Snacks       0.88      0.82      0.85        73
         Automotive Products       1.00      0.16      0.27        19
                        Beer       0.96      0.95      0.96       160
                       Candy       0.76      0.97      0.85       337
                  Cigarettes       0.96      0.99      0.97        75
    Cold Dispensed Beverages       0.00      0.00      0.00         4
       Commissary - Packaged       1.00      0.58      0.74        36
                E-Cigarettes       1.00      0.84      0.91        37
              Edible Grocery       0.70      0.64      0.67       181
         Fluid Milk Products       0.00      0.00      0.00         6
                 Foodservice       0.00      0.00      0.00        15
  Frozen Dispensed Beverages       0.00      0.00      0.00         3
                Frozen Foods       0.00      0.00      0.00         1
         General Me

C:\Users\rmr\.conda\envs\class\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


<h1>K-nearest Neighbor</h1>

In [10]:
from sklearn.neighbors import KNeighborsClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', KNeighborsClassifier()),
                     ])

text_clf.fit(X_train, y_train)

predicted = text_clf.predict(X_test)

print(metrics.classification_report(y_test, predicted))

                              precision    recall  f1-score   support

          Alternative Snacks       0.74      0.99      0.85        73
         Automotive Products       0.88      0.74      0.80        19
                        Beer       0.93      0.94      0.93       160
                       Candy       0.88      0.91      0.90       337
                  Cigarettes       0.89      1.00      0.94        75
    Cold Dispensed Beverages       0.75      0.75      0.75         4
       Commissary - Packaged       0.82      0.89      0.85        36
                E-Cigarettes       0.92      0.89      0.90        37
              Edible Grocery       0.82      0.65      0.72       181
         Fluid Milk Products       0.83      0.83      0.83         6
                 Foodservice       0.58      0.47      0.52        15
  Frozen Dispensed Beverages       1.00      0.33      0.50         3
                Frozen Foods       1.00      1.00      1.00         1
         General Me

C:\Users\rmr\.conda\envs\class\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


<h1>Support Vector Machine (SVM)</h1>

In [11]:
from sklearn.svm import LinearSVC

In [12]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LinearSVC()),
                     ])

text_clf.fit(X_train, y_train)


predicted = text_clf.predict(X_test)

print(metrics.classification_report(y_test, predicted))

                              precision    recall  f1-score   support

          Alternative Snacks       0.90      1.00      0.95        73
         Automotive Products       1.00      0.74      0.85        19
                        Beer       0.99      0.97      0.98       160
                       Candy       0.95      0.96      0.96       337
                  Cigarettes       1.00      1.00      1.00        75
    Cold Dispensed Beverages       1.00      1.00      1.00         4
       Commissary - Packaged       0.97      0.86      0.91        36
                E-Cigarettes       0.97      0.97      0.97        37
              Edible Grocery       0.78      0.76      0.77       181
         Fluid Milk Products       0.83      0.83      0.83         6
                 Foodservice       0.87      0.87      0.87        15
  Frozen Dispensed Beverages       1.00      0.67      0.80         3
                Frozen Foods       1.00      1.00      1.00         1
         General Me

<h1>Decision Tree</h1>

In [13]:
from sklearn import tree
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', tree.DecisionTreeClassifier()),
                     ])

text_clf.fit(X_train, y_train)


predicted = text_clf.predict(X_test)

print(metrics.classification_report(y_test, predicted))

                              precision    recall  f1-score   support

          Alternative Snacks       0.83      0.89      0.86        73
         Automotive Products       0.53      0.42      0.47        19
                        Beer       0.93      0.87      0.90       160
                       Candy       0.75      0.90      0.82       337
                  Cigarettes       0.96      0.99      0.97        75
    Cold Dispensed Beverages       1.00      0.75      0.86         4
       Commissary - Packaged       0.94      0.81      0.87        36
                E-Cigarettes       0.89      0.86      0.88        37
              Edible Grocery       0.66      0.59      0.63       181
         Fluid Milk Products       0.40      0.33      0.36         6
                 Foodservice       0.50      0.60      0.55        15
  Frozen Dispensed Beverages       1.00      0.67      0.80         3
                Frozen Foods       0.50      1.00      0.67         1
         General Me

C:\Users\rmr\.conda\envs\class\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


<h1>Random Forest</h1>

In [14]:
from sklearn.ensemble import RandomForestClassifier

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', RandomForestClassifier(n_estimators=100)),
                     ])

text_clf.fit(X_train, y_train)


predicted = text_clf.predict(X_test)

print(metrics.classification_report(y_test, predicted))

                              precision    recall  f1-score   support

          Alternative Snacks       0.87      0.93      0.90        73
         Automotive Products       0.91      0.53      0.67        19
                        Beer       0.94      0.94      0.94       160
                       Candy       0.78      0.97      0.86       337
                  Cigarettes       1.00      1.00      1.00        75
    Cold Dispensed Beverages       1.00      0.50      0.67         4
       Commissary - Packaged       1.00      0.72      0.84        36
                E-Cigarettes       0.94      0.92      0.93        37
              Edible Grocery       0.74      0.62      0.67       181
         Fluid Milk Products       0.56      0.83      0.67         6
                 Foodservice       0.77      0.67      0.71        15
  Frozen Dispensed Beverages       1.00      0.67      0.80         3
                Frozen Foods       1.00      1.00      1.00         1
         General Me

<h1>Deep Neural Networks</h1>

In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import  Dropout, Dense
from tensorflow.keras.models import Sequential
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
from sklearn import metrics

In [17]:
def TFIDF(X_train, X_test,MAX_NB_WORDS=75000):
    vectorizer_x = TfidfVectorizer(max_features=MAX_NB_WORDS)
    X_train = vectorizer_x.fit_transform(X_train).toarray()
    X_test = vectorizer_x.transform(X_test).toarray()
    print("tf-idf with",str(np.array(X_train).shape[1]),"features")
    return (X_train,X_test)

In [18]:
#prepare target
def prepare_targets_le(y_train, y_test):
    # need to make sure we force a 2D array or we'll run into trouble with LE
    y_train = y_train.to_numpy().reshape(-1,1)
    y_test = y_test.to_numpy().reshape(-1,1)
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_test_enc = le.transform(y_test)
    return y_train_enc, y_test_enc

In [19]:
#prepare target
def prepare_targets_oe(y_train, y_test):
    oe = OrdinalEncoder()
    oe.fit(y_train)
    y_train_enc = oe.transform(y_train)
    y_test_enc = oe.transform(y_test)
    return y_train_enc, y_test_enc

In [64]:
def Build_Model_DNN_Text(shape, nClasses, dropout=0.5):
    """
    buildModel_DNN_Tex(shape, nClasses,dropout)
    Build Deep neural networks Model for text classification
    Shape is input feature space
    nClasses is number of classes
    """
    model = Sequential()
    node = 2500 # number of nodes
    nLayers = 1 # number of  hidden layer

    model.add(Dense(node,input_dim=shape,activation='relu'))
    model.add(Dropout(dropout))
    for i in range(0,nLayers):
        model.add(Dense(node,input_dim=node,activation='relu'))
        model.add(Dropout(dropout))
    model.add(Dense(nClasses, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [21]:
df_sku['Category'].nunique()


29

In [31]:
import numpy as np
#y_train = y_train.values.reshape(-1,1)
#y_test = y_test.values.reshape(-1,1)

In [32]:
X_train_tfidf,X_test_tfidf = TFIDF(X_train,X_test)
y_train_enc, y_test_enc = prepare_targets_oe(y_train, y_test)

tf-idf with 5125 features


In [41]:
y_test_enc = y_test_enc.astype(float)

In [45]:
y_test_enc = y_test_enc.flatten()

In [66]:
y_test_enc.shape

(2013, 1)

In [67]:
predicted.shape

(2013, 29)

In [61]:
#[i for i in y_test_enc ]

In [62]:
pred = np.argmax(predicted,axis=1)

In [63]:
predicted = model_DNN.predict(X_test_tfidf)

print(metrics.classification_report(y_test_enc, pred))

              precision    recall  f1-score   support

           0       0.90      0.90      0.90        73
           1       0.79      0.79      0.79        19
           2       0.97      0.96      0.97       160
           3       0.97      0.95      0.96       337
           4       1.00      1.00      1.00        75
           5       1.00      1.00      1.00         4
           6       0.97      0.92      0.94        36
           7       0.97      0.97      0.97        37
           8       0.76      0.77      0.77       181
           9       1.00      1.00      1.00         6
          10       0.70      0.93      0.80        15
          11       1.00      0.67      0.80         3
          12       1.00      1.00      1.00         1
          13       0.84      0.88      0.86       127
          14       0.97      0.83      0.90       109
          15       0.20      1.00      0.33         1
          16       1.00      1.00      1.00         1
          17       1.00    

In [65]:
X_train_tfidf,X_test_tfidf = TFIDF(X_train,X_test)
y_train_enc, y_test_enc = prepare_targets_oe(y_train, y_test)
model_DNN = Build_Model_DNN_Text(X_train_tfidf.shape[1], 29) # 29 is df_sku['Category'].nunique()
model_DNN.fit(X_train_tfidf, y_train_enc,
                              validation_data=(X_test_tfidf, y_test_enc),
                              epochs=10,
                              batch_size=128,
                              verbose=2)

predicted = model_DNN.predict(X_test_tfidf)

print(metrics.classification_report(y_test, predicted))

tf-idf with 5125 features
Train on 8052 samples, validate on 2013 samples
Epoch 1/10
8052/8052 - 12s - loss: 1.8317 - accuracy: 0.5067 - val_loss: 0.7619 - val_accuracy: 0.8058
Epoch 2/10
8052/8052 - 11s - loss: 0.4581 - accuracy: 0.8884 - val_loss: 0.4500 - val_accuracy: 0.8917
Epoch 3/10
8052/8052 - 11s - loss: 0.1756 - accuracy: 0.9540 - val_loss: 0.4221 - val_accuracy: 0.9016
Epoch 4/10
8052/8052 - 11s - loss: 0.0840 - accuracy: 0.9778 - val_loss: 0.4399 - val_accuracy: 0.9046
Epoch 5/10
8052/8052 - 12s - loss: 0.0560 - accuracy: 0.9848 - val_loss: 0.4608 - val_accuracy: 0.9021
Epoch 6/10
8052/8052 - 12s - loss: 0.0365 - accuracy: 0.9902 - val_loss: 0.4708 - val_accuracy: 0.9061
Epoch 7/10
8052/8052 - 11s - loss: 0.0331 - accuracy: 0.9907 - val_loss: 0.4821 - val_accuracy: 0.9021
Epoch 8/10
8052/8052 - 12s - loss: 0.0308 - accuracy: 0.9904 - val_loss: 0.4818 - val_accuracy: 0.9066
Epoch 9/10
8052/8052 - 12s - loss: 0.0193 - accuracy: 0.9938 - val_loss: 0.5191 - val_accuracy: 0.8992

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

In [108]:
def prepare_inputs(X_train, X_test):
    oe = OrdinalEncoder()
    oe.fit(X_train)
    X_train_enc = oe.transform(X_train)
    X_test_enc = oe.transform(X_test)
    return X_train_enc, X_test_enc

In [109]:
# prepare target
def prepare_targets(y_train, y_test):
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_test_enc = le.transform(y_test)
    return y_train_enc, y_test_enc

In [110]:
# prepare input data
X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)
# prepare output data
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)

ValueError: Expected 2D array, got 1D array instead:
array=['HALLS C DROP ICE BL(ICE PEP)' 'BIC UGLY SWEATERS SE'
 'BRUTON 6 OZ PLS GLASS' ... 'LAYS POTATO CHIP SALT AND VINEGAR'
 'KETTLE THICK&BOLD BBQ CHP' 'MCCORMICK BLACK PEPPER 1.5Z'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
# define the model
model = Sequential()
model.add(Dense(10, input_dim=X_train_enc.shape[1], activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train_enc, y_train_enc, epochs=100, batch_size=16, verbose=2)
# evaluate the keras model
_, accuracy = model.evaluate(X_test_enc, y_test_enc, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))